In [1]:
# Import libraries
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

from tqdm import tqdm

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow is using:", tf.test.gpu_device_name())
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        print(e)

2025-06-17 23:36:45.787119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750228605.884196     804 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750228605.908530     804 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-17 23:36:46.126347: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available: 1
TensorFlow is using: /device:GPU:0
GPU memory growth enabled.


I0000 00:00:1750228612.076057     804 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1750228612.077633     804 gpu_device.cc:2022] Created device /device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:06:00.0, compute capability: 8.6


In [2]:
def load_data(spectrogram_dir, label_dir):
    spectrograms = []
    labels = []

    # Load spectrograms
    spectrogram_files = sorted(os.listdir(spectrogram_dir))
    label_files = sorted(os.listdir(label_dir))

    total_files = len(spectrogram_files)

    for spec_file, label_file in tqdm(zip(spectrogram_files, label_files), 
                                        total=total_files, 
                                        desc="Loading Data", 
                                        unit="file"):
        # Load data
        spec_data = np.load(os.path.join(spectrogram_dir, spec_file)) # (n, 128, 42)
        label_data = np.load(os.path.join(label_dir, label_file)).T   # (n, 5)

        # Select multiple columns
        label_data = label_data[:, [0, 1, 2, 3, 4]]  # (n, 5)

        # Append if data is valid
        if len(spec_data) > 0:
            spectrograms.append(spec_data)
            labels.append(label_data)

    # Convert to numpy arrays 
    if spectrograms:
        X = np.vstack(spectrograms)
        Y = np.vstack(labels)
        return X, Y
    else:
        return np.array([]), np.array([])

In [3]:
X_train, Y_train = load_data("spectrograms_train", "labels_train")
X_train = np.expand_dims(X_train, axis=-1)
print("Final X_train:", X_train.shape, "Final Y_train:", Y_train.shape)

Loading Data: 100%|██████████| 1289/1289 [00:17<00:00, 72.02file/s]


Final X_train: (319508, 128, 42, 1) Final Y_train: (319508, 5)


In [4]:
# Occurrences
class_counts = np.sum(Y_train, axis=0)
total_samples = len(Y_train)

for i, count in enumerate(class_counts):
    print(f"Class {i}: {count} occurrences ({count / total_samples:.2%} of the data)")

Class 0: 128695 occurrences (40.28% of the data)
Class 1: 246807 occurrences (77.25% of the data)
Class 2: 257671 occurrences (80.65% of the data)
Class 3: 155212 occurrences (48.58% of the data)
Class 4: 271620 occurrences (85.01% of the data)


In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001), input_shape=(128, 42, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001)),
    Dropout(0.4),
    Dense(5, activation='sigmoid', dtype='float32')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

/home/garagemoney/rtii/my_venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1750228865.121678     804 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:06:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 42, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 21, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 5, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10240)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,310,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,404,165 (5.36 MB)

 Trainable params: 1,404,165 (5.36 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train model
history = model.fit(X_train, Y_train, batch_size=64, epochs=5, verbose=1)

# Final loss & accuracy
final_loss = history.history['loss'][-1]
final_accuracy = history.history['binary_accuracy'][-1]
print(f"Final Loss: {final_loss:.4f}, Final Accuracy: {final_accuracy:.4f}")

2025-06-17 23:41:28.159333: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6870700032 exceeds 10% of free system memory.
2025-06-17 23:42:26.301634: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6870700032 exceeds 10% of free system memory.


Epoch 1/5


I0000 00:00:1750228968.961909    2061 service.cc:148] XLA service 0x7f5ebc0046d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750228968.963690    2061 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-06-17 23:42:49.308538: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750228969.629176    2061 cuda_dnn.cc:529] Loaded cuDNN version 90300


   6/4993 ━━━━━━━━━━━━━━━━━━━━ 1:46 21ms/step - binary_accuracy: 0.5442 - loss: 2.8597

I0000 00:00:1750228974.974593    2061 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4993/4993 ━━━━━━━━━━━━━━━━━━━━ 103s 19ms/step - binary_accuracy: 0.8455 - loss: 0.3924
Epoch 2/5
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 91s 18ms/step - binary_accuracy: 0.8979 - loss: 0.2771
Epoch 3/5
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 93s 19ms/step - binary_accuracy: 0.9069 - loss: 0.2620
Epoch 4/5
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 92s 19ms/step - binary_accuracy: 0.9108 - loss: 0.2556
Epoch 5/5
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 91s 18ms/step - binary_accuracy: 0.9129 - loss: 0.2518
Final Loss: 0.2528, Final Accuracy: 0.9128


In [7]:
# Get predictions
Y_pred = model.predict(X_train)

# Convert to binary (threshold the probabilities)
threshold = 0.5
Y_pred_binary = (Y_pred > threshold).astype(int)

# Calculate metrics
precision = precision_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
recall = recall_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
f1 = f1_score(Y_train, Y_pred_binary, average='samples', zero_division=0)

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

2025-06-17 23:51:29.646175: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6870700032 exceeds 10% of free system memory.
2025-06-17 23:51:40.607212: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6870700032 exceeds 10% of free system memory.


9985/9985 ━━━━━━━━━━━━━━━━━━━━ 49s 5ms/step
Precision: 0.9141
Recall: 0.8886
F1-Score: 0.8929


In [8]:
X_test, Y_test = load_data("spectrograms_test", "labels_test")
X_test = np.expand_dims(X_test, axis=-1)
Y_pred_test = model.predict(X_test)

Loading Data:   0%|          | 0/151 [00:00<?, ?file/s]

Loading Data: 100%|██████████| 151/151 [00:01<00:00, 85.04file/s]


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [11]:
# --- Threshold Adjustment ---
num_classes = Y_test.shape[1]
thresholds = np.array([0.5] * num_classes)  # Default 0.5
# Adjust thresholds as needed (example):
thresholds[0] = 0.4
thresholds[1] = 0.6
thresholds[2] = 0.5
thresholds[3] = 0.3
thresholds[4] = 0.5

Y_pred_test_binary = (Y_pred_test >= thresholds).astype(int)
# --- End Threshold Adjustment ---

precision_test = precision_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
recall_test = recall_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
f1_test = f1_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)

print(f"Test Precision: {precision_test:.4f}")
print(f"Test Recall: {recall_test:.4f}")
print(f"Test F1-Score: {f1_test:.4f}")

# Compute per-class precision, recall, and F1-score
precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
    Y_test, Y_pred_test_binary, average=None, zero_division=0
)

# Occurrences
class_counts = np.sum(Y_train, axis=0) # Make sure Y_train is defined.
total_samples = len(Y_train)

for i in range(num_classes):
    print(f"Class {i} | Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, "
          f"F1={f1_per_class[i]:.4f} | Thresh={thresholds[i]:.1f}, Occurrences={int(class_counts[i])} ({class_counts[i] / total_samples:.2%})")

Test Precision: 0.9042
Test Recall: 0.8747
Test F1-Score: 0.8779
Class 0 | Precision=0.8666, Recall=0.7176, F1=0.7851 | Thresh=0.4, Occurrences=128695 (40.28%)
Class 1 | Precision=0.9242, Recall=0.9186, F1=0.9214 | Thresh=0.6, Occurrences=246807 (77.25%)
Class 2 | Precision=0.9678, Recall=0.9641, F1=0.9659 | Thresh=0.5, Occurrences=257671 (80.65%)
Class 3 | Precision=0.8958, Recall=0.8420, F1=0.8681 | Thresh=0.3, Occurrences=155212 (48.58%)
Class 4 | Precision=0.9746, Recall=0.9751, F1=0.9748 | Thresh=0.5, Occurrences=271620 (85.01%)


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 42, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 21, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 5, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10240)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,310,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,212,497 (16.07 MB)

 Trainable params: 1,404,165 (5.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,808,332 (10.71 MB)

In [16]:
model.export("saved_model_dir")

model.save("instrument_classifier.h5")
model = tf.keras.models.load_model("instrument_classifier.h5", compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("instrument_classifier.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: saved_model_dir/assets


INFO:tensorflow:Assets written to: saved_model_dir/assets


Saved artifact at 'saved_model_dir'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 42, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  140047802301456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802368944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802371408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802370000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802379680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802372288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802381616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802377040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802381792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140047802378624: TensorSpec(shape=(), dtype=tf.resource, name=None)


INFO:tensorflow:Assets written to: /tmp/tmp4dchnrgp/assets


INFO:tensorflow:Assets written to: /tmp/tmp4dchnrgp/assets


Saved artifact at '/tmp/tmp4dchnrgp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 42, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  140045734087488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140046672874480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045536400784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045536410992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045537057376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045537058080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045537064416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045535169872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045535177792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140045535175856: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1750229958.476759     804 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1750229958.476928     804 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-17 23:59:18.479609: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4dchnrgp
2025-06-17 23:59:18.480153: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-17 23:59:18.480166: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp4dchnrgp
I0000 00:00:1750229958.487014     804 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-06-17 23:59:18.488013: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-17 23:59:18.530946: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp4dchnrgp
2025-06-17 23:59:18.539595: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 